# StudentLayer Data Cleaning

Let's explore the dataset and clean it up to make sure that it is fit-for-purpose.

We'll start by importing the necessary libraries. We'll be using *pandas* to analyse and manipulate our dataset

In [146]:
import json, pandas
pandas.options.display.max_rows = 10

## Importing the data

We can open our JSON file in *pandas* using the *read_json* function. This will convert our JSON file to a pandas dataframe.

In [147]:
df = pandas.read_json('universities.json')
df.dtypes

web_pages         object
name              object
alpha_two_code    object
state-province    object
domains           object
country           object
dtype: object

Immediately we can notice a problem - the JSON fields that were arrays (web_pages and domains) have been imported as an array within each row. This will make it hard to query these columns as we analyse and clean the data. 

Fortunately, *pandas* provides a functionc called *json_normalize* that we can use to flatten our nested JSON data into a flat table. 

The *json_normalize* takes a Python JSON object as it's main argument so we'll have to load our JSON file in manually.

In [148]:
with open('universities.json') as file:
    universities = json.loads(file.read())

Now we can normalize our JSON object into a flat table. We'll use the *record_path* argument point to where to create the table from.

In [149]:
df = pandas.json_normalize(universities, record_path=['domains'])
df

,0
0,marywood.edu
1,cstj.qc.ca
2,lindenwood.edu
3,davietjal.org
4,lpu.in
...,...
9913,al.infnet.edu.br
9914,prof.infnet.edu.br
9915,ssst.edu.ba
9916,hbc.edu.cn


Wait a second, what's happend to the other columns? Well, we need to add these back in as 'metadata' for each row using the *metadata* argument, naming all the columns we want. 

In [150]:
df = pandas.json_normalize(universities, record_path=['domains'], meta=['name', 'alpha_two_code', 'country'])
df

,0,name,alpha_two_code,country
0,marywood.edu,Marywood University,US,United States
1,cstj.qc.ca,Cégep de Saint-Jérôme,CA,Canada
2,lindenwood.edu,Lindenwood University,US,United States
3,davietjal.org,DAV Institute of Engineering & Technology,IN,India
4,lpu.in,Lovely Professional University,IN,India
...,...,...,...,...
9913,al.infnet.edu.br,Instituto Infnet,BR,Brazil
9914,prof.infnet.edu.br,Instituto Infnet,BR,Brazil
9915,ssst.edu.ba,Sarajevo School of Science and Technology,BA,Bosnia and Herzegovina
9916,hbc.edu.cn,Hubei Business College,CN,China


This is much better! We're missing a proper column name for the domains, so let's rename this.

In [151]:
df = df.rename(columns={0: "domain"})
df

,domain,name,alpha_two_code,country
0,marywood.edu,Marywood University,US,United States
1,cstj.qc.ca,Cégep de Saint-Jérôme,CA,Canada
2,lindenwood.edu,Lindenwood University,US,United States
3,davietjal.org,DAV Institute of Engineering & Technology,IN,India
4,lpu.in,Lovely Professional University,IN,India
...,...,...,...,...
9913,al.infnet.edu.br,Instituto Infnet,BR,Brazil
9914,prof.infnet.edu.br,Instituto Infnet,BR,Brazil
9915,ssst.edu.ba,Sarajevo School of Science and Technology,BA,Bosnia and Herzegovina
9916,hbc.edu.cn,Hubei Business College,CN,China


### What happend to web_pages?

You might have noticed that we've dropped the *web_pages* column/field by not naming it as a 'meta' column when calling *json_normalize*. This is intentional. 

The main reason is that this field is mostly redundant, given that we have the domain, and not particuarly useful for our problem of verifiying student emails. Potentially this field could be useful for identifying subdomains of a particular university domain (i.e. different departments), but it is highly unlikely that the data in this field will capture all webpages and/or subdomains, making it more effect that it is worth. 

We'll get rid of it for the time being.

### What happened to state-province?

We've also dropped the *state-province* columns for a similar reason to *web_pages*. In this case, this column is more effort that it's worth because it is empty (or *None*) for many rows. Furthermore, when it comes to combining entries later, trying to match *state-province* to the relevant country will be tricky. 

### What happened to country?

Lastly, you might also notice we've dropped the country column. This is purely for brevity, as we already have the alpha-two-code - a much more accurate system for denote countries than string names. 

## Sorting

To make our lives easier momentarily, we'll want to sort our rows by a column. We'll use *name*, as it should be the unique value that distinguishes a particular university. Why not *domain*? We saw earlier that each university had an array of **at least one** domain. This means that a university could have multiple domais that point to it, making domains a poor primary key. 

*Note: This is notwithstanding duplicate universities, which is an issue we will tackle shortly*.

Let's sort the dataframe by the values in the *name* column

In [152]:
df = df.sort_values('name')
df

,domain,name,alpha_two_code,country
7222,uab.ro,1 December University of Alba Iulia,RO,Romania
2607,smmu.edu.cn,2nd Military Medical University,CN,China
2643,tmmu.edu.cn,3rd Military Medical University,CN,China
3203,student.42.fr,42 FR,FR,France
66,student.42.us.org,42 US,US,United States
...,...,...,...,...
2148,etsmtl.ca,"École de technologie supérieure, Université du...",CA,Canada
2151,hec.ca,École des Hautes Études Commerciales,CA,Canada
2147,enap.uquebec.ca,"École nationale d'administration publique, Uni...",CA,Canada
3415,cpe.fr,"École supérieure de chimie, physique, électron...",FR,France


We now need to reset the dataframe's index to reflect this new order.

In [153]:
df = df.reset_index(drop=True)
df

,domain,name,alpha_two_code,country
0,uab.ro,1 December University of Alba Iulia,RO,Romania
1,smmu.edu.cn,2nd Military Medical University,CN,China
2,tmmu.edu.cn,3rd Military Medical University,CN,China
3,student.42.fr,42 FR,FR,France
4,student.42.us.org,42 US,US,United States
...,...,...,...,...
9913,etsmtl.ca,"École de technologie supérieure, Université du...",CA,Canada
9914,hec.ca,École des Hautes Études Commerciales,CA,Canada
9915,enap.uquebec.ca,"École nationale d'administration publique, Uni...",CA,Canada
9916,cpe.fr,"École supérieure de chimie, physique, électron...",FR,France


That's great, but for aesethetics let's also reorder the columns.

In [154]:
df = df[["name", "domain", "alpha_two_code"]]
df

,name,domain,alpha_two_code
0,1 December University of Alba Iulia,uab.ro,RO
1,2nd Military Medical University,smmu.edu.cn,CN
2,3rd Military Medical University,tmmu.edu.cn,CN
3,42 FR,student.42.fr,FR
4,42 US,student.42.us.org,US
...,...,...,...
9913,"École de technologie supérieure, Université du...",etsmtl.ca,CA
9914,École des Hautes Études Commerciales,hec.ca,CA
9915,"École nationale d'administration publique, Uni...",enap.uquebec.ca,CA
9916,"École supérieure de chimie, physique, électron...",cpe.fr,FR


## Duplicates

We need to check if there are any duplicate records.

Let's first check for rows that are complete duplicates (i.e. all rows)

In [155]:
df[df.duplicated(keep=False)].size

0

That's good news. Let's check for duplicate rows with that have same *name* and *country* to hopefully find multiple domains the point to the same university. 

In [156]:
duplicated_name_country = df[df.duplicated(subset=["name", "alpha_two_code"], keep=False)]
duplicated_name_country

,name,domain,alpha_two_code
15,Aalborg University,auc.dk,DK
16,Aalborg University,student.aau.dk,DK
17,Aalborg University,aau.dk,DK
41,Academy of Art University,academyart.edu,US
42,Academy of Art University,art.edu,US
...,...,...,...
9773,Xihua University,xhu.edu.cn,CN
9774,Xihua University,stu.xhu.edu.cn,CN
9775,Xihua University,mail.xhu.edu.cn,CN
9910,École Polytechnique,polytechnique.fr,FR


So we now know that some universities have multiple domains. Now let's combine these entries. We can create a *groupby* object and the *agg* method to define how we want to aggregate each column. In this case, as it's the *name* and *alpha_two_code* that duplicated, we use a lambda function to aggregate the various *domain* values for these rows into one list. Note we have to specify that we want the 'first' alpha_two_code for each name - which one we use doesn't matter as we known it will be the same, but we have to choose from 'first' or 'last'

In [157]:
grouped_name_country = duplicated_name_country.groupby(['name']).agg({
    'domain': lambda x: list(x),
    'alpha_two_code': 'first',
})
grouped_name_country = grouped_name_country.reset_index()
grouped_name_country

,name,domain,alpha_two_code
0,Aalborg University,"[auc.dk, student.aau.dk, aau.dk]",DK
1,Academy of Art University,"[academyart.edu, art.edu]",US
2,Augusta University,"[gru.edu, augusta.edu]",US
3,Blue Ridge Community College,"[blueridge.edu, brcc.edu]",US
4,Box Hill Institute,"[bhtafe.edu.au, boxhill.edu.au]",AU
...,...,...,...
160,Wilfrid Laurier University,"[wlu.ca, mylaurier.ca]",CA
161,Wirtschaftsuniversität Wien,"[wu.ac.at, wu-wien.ac.at]",AT
162,Wonkwang University,"[wku.ac.kr, wonkwang.ac.kr]",KR
163,Xihua University,"[xhu.edu.cn, stu.xhu.edu.cn, mail.xhu.edu.cn]",CN


But if it's possible for rows to share the same *name* and *alpha_two_code* and have different domains, is it possible for rows to share the same *name* and *domain*, but have a different *alpha_two_code*? Let's find out.

In [158]:
duplicated_name_domain = df[df.duplicated(subset=["name", "domain"], keep=False)]
duplicated_name_domain

,name,domain,alpha_two_code
101,Aga Khan University,aku.edu,GB
102,Aga Khan University,aku.edu,KE
103,Aga Khan University,aku.edu,UG
104,Aga Khan University,aku.edu,PK
105,Aga Khan University,aku.edu,TZ
...,...,...,...
9384,Université des Antilles et de la Guyane,univ-ag.fr,GF
9385,Université des Antilles et de la Guyane,univ-ag.fr,MQ
9386,Université des Antilles et de la Guyane,univ-ag.fr,GP
9568,Warnborough University,warnborough.edu,GB


It is! This would appear to be universities that have multiple campuses internationally. Now these universities may often have different names (i.e. *University, X Country and University, Y Country*), however this will not be a suitable approach for us. Firstly, this would probably entail a lot of manual work. Secondly, and more importantly, it doesn't matter - with only one domain we don't know which specific campus a domain is pointing to. Therefore the best approach will be to just combine these countries into a list.

Now we can combine these entries too. We'll use a similar method as before, but this time we'll change our aggregate function.

In [159]:
grouped_name_domain = duplicated_name_domain.groupby(['name']).agg({
    'domain': 'first',
    'alpha_two_code': lambda x: list(x),
})
grouped_name_domain = grouped_name_domain.reset_index()
grouped_name_domain

,name,domain,alpha_two_code
0,Aga Khan University,aku.edu,"[GB, KE, UG, PK, TZ]"
1,University of Guam,uog.edu,"[US, GU]"
2,University of the South Pacific,usp.ac.fj,"[SB, FJ]"
3,University of the Virgin Islands,uvi.edu,"[US, VG]"
4,Université des Antilles et de la Guyane,univ-ag.fr,"[GF, MQ, GP]"
5,Warnborough University,warnborough.edu,"[GB, IE]"


At this point, we should also check whether there is an intersection between these two (sub-)dataframes, as this will make things more complicated. 

In [160]:
duplicated_intersection = pandas.merge(duplicated_name_domain, duplicated_name_country, how='inner', on=['domain'])
duplicated_intersection.size

0

Phew!

## Dropping duplicates

Now that we've managed to combine these duplicate rows, we should drop them from our initial dataframe and add back in our aggregated rows. Let's start by dropping the duplicate rows

In [161]:
df = df.drop_duplicates(subset=["name", "alpha_two_code"], keep=False)
df = df.drop_duplicates(subset=["name", "domain"], keep=False)

Now before we add back in our aggregated rows, let's have a look and see if there are any duplicated universities remaining.

In [162]:
df[df.duplicated(subset=["name"], keep=False)].sort_values("name")

,name,domain,alpha_two_code
285,American University,aubih.ba,BA
286,American University,american.edu,US
363,Applied Science University,asu.edu.bh,BH
364,Applied Science University,asu.edu.jo,JO
371,Arab Open University,aou.org.bh,BH
...,...,...,...
9627,West Coast University (WCU),westcoastuniversity.bz,BZ
9667,Western University,wu.edu.az,AZ
9668,Western University,western.edu.kh,KH
9758,Xavier University,xu.edu.ph,PH


There are! What's going on? Well if we loook at the data we can see that both the domains (and countries) for these are different. From our perspective, using domains as our key, this will be ok. What is probably happening is that many of these universities will be coincidental duplicates - nothing we can do about that!

But what about duplicate domains?

In [163]:
df[df.duplicated(subset=["domain"], keep=False)].sort_values("domain")

,name,domain,alpha_two_code
3035,Institut National Supérieur de Formation Agro-...,agro.roazhon.inra.fr,FR
1788,Ecole Nationale Supérieure d'Agronomie de Rennes,agro.roazhon.inra.fr,FR
11,"AKAD Hochschulen für Berufstätige, Fachhochsch...",akad.de,DE
2774,Hochschule für Berufstätige Rendsburg,akad.de,DE
5894,San Antonio College,alamo.edu,US
...,...,...,...
9586,Washington State University at Tri-Cities,wsu.edu,US
6653,Tarlac State University,www2.mozcom.com,PH
1030,Central Luzon State University,www2.mozcom.com,PH
9809,Yarmouk University,yu.edu.jo,JO


Again we still have duplicates, however this is a more perculiar case. Multiple universities pointed to by the same domain. Some in the same country, others in different countries. These seem to be affiliate institutions, but without the same consistency that we had before. There's nothing we can really do about this. 

Now we can append our aggregated rows to our dataframe. But first we need to convert the domain and alpha_two_code columns to lists of strings for consistency.

In [164]:
df['domain'] = df['domain'].apply(lambda x: [x])
df['alpha_two_code'] = df['alpha_two_code'].apply(lambda x: [x])
df

,name,domain,alpha_two_code
0,1 December University of Alba Iulia,[uab.ro],[RO]
1,2nd Military Medical University,[smmu.edu.cn],[CN]
2,3rd Military Medical University,[tmmu.edu.cn],[CN]
3,42 FR,[student.42.fr],[FR]
4,42 US,[student.42.us.org],[US]
...,...,...,...
9913,"École de technologie supérieure, Université du...",[etsmtl.ca],[CA]
9914,École des Hautes Études Commerciales,[hec.ca],[CA]
9915,"École nationale d'administration publique, Uni...",[enap.uquebec.ca],[CA]
9916,"École supérieure de chimie, physique, électron...",[cpe.fr],[FR]


Now let's append the aggregated rows.

In [165]:
df = df.append(grouped_name_domain)
df = df.append(grouped_name_country)
df

,name,domain,alpha_two_code
0,1 December University of Alba Iulia,[uab.ro],[RO]
1,2nd Military Medical University,[smmu.edu.cn],[CN]
2,3rd Military Medical University,[tmmu.edu.cn],[CN]
3,42 FR,[student.42.fr],[FR]
4,42 US,[student.42.us.org],[US]
...,...,...,...
160,Wilfrid Laurier University,"[wlu.ca, mylaurier.ca]",CA
161,Wirtschaftsuniversität Wien,"[wu.ac.at, wu-wien.ac.at]",AT
162,Wonkwang University,"[wku.ac.kr, wonkwang.ac.kr]",KR
163,Xihua University,"[xhu.edu.cn, stu.xhu.edu.cn, mail.xhu.edu.cn]",CN


Now we just need to reset our index to reflect this new dataframe.

In [166]:
df = df.sort_values("name")
df.reset_index(drop=True)

,name,domain,alpha_two_code
0,1 December University of Alba Iulia,[uab.ro],[RO]
1,2nd Military Medical University,[smmu.edu.cn],[CN]
2,3rd Military Medical University,[tmmu.edu.cn],[CN]
3,42 FR,[student.42.fr],[FR]
4,42 US,[student.42.us.org],[US]
...,...,...,...
9721,"École de technologie supérieure, Université du...",[etsmtl.ca],[CA]
9722,École des Hautes Études Commerciales,[hec.ca],[CA]
9723,"École nationale d'administration publique, Uni...",[enap.uquebec.ca],[CA]
9724,"École supérieure de chimie, physique, électron...",[cpe.fr],[FR]


That's it all we need to do now is export our dataframe back to a JSON file:

In [167]:
output = json.loads(df.to_json(orient='records', force_ascii=False))

In [168]:
with open ('universities_clean.json', 'w') as file_handle:
    json.dump(output, file_handle, ensure_ascii=False, indent=2)

That's it! Well done if you made it to this point!